In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 103.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 92.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=8895ef619604677a4dd3c84a905b2a9565b60ff9c61362aa0e098929cdbf71b6
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00


In [ ]:

"""
MODIFIED: (efv) Use STSb-multi-mt Spanish
source: https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/sts/training_stsbenchmark.py

---

This examples trains BERT (or any other transformer model like RoBERTa, DistilBERT etc.) for the STSbenchmark from scratch. It generates sentence embeddings
that can be compared using cosine-similarity to measure the similarity.

Usage:
python training_nli.py

OR
python training_nli.py pretrained_transformer_model_name
"""
from torch.utils.data import DataLoader
from torch import cuda
import math
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import sys
import os
import gzip
import csv

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

logging.info(f"CUDA Device Name:{cuda.get_device_name()}")

In [ ]:
from datasets import DatasetDict

In [ ]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="/content/drive/MyDrive/nepali_embeddings_data/main_final.csv")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 5199
    })
})

In [ ]:
train_test = dataset["train"].train_test_split(test_size=0.1)

In [ ]:
test_valid = train_test["test"].train_test_split(test_size=0.5)


In [ ]:
train_test_valid_dataset = DatasetDict({
    'train': train_test['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

In [ ]:
train_test_valid_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 4679
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 260
    })
    valid: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 260
    })
})

In [ ]:

MODEL = 'Rajan/NepaliBERT'

#You can specify any huggingface/transformers pre-trained model here, for example, bert-base-uncased, roberta-base, xlm-roberta-base
#model_name = sys.argv[1] if len(sys.argv) > 1 else 'distilbert-base-uncased'
model_name = MODEL

# Read the dataset
train_batch_size = 16
num_epochs = 4
model_save_path = '/content/drive/MyDrive/nepali_embeddings_data/model/'+model_name.replace("/", "-")+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Use Huggingface/transformers model (like BERT, RoBERTa, XLNet, XLM-R) for mapping tokens to embeddings
word_embedding_model = models.Transformer(model_name)

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# Convert the dataset to a DataLoader ready for training
logging.info("Read stsb-multi-mt train dataset")

train_samples = []
dev_samples = []
test_samples = []

def samples_from_dataset(dataset):
    samples = [InputExample(texts=[e['sentence1'], e['sentence2']], label=e['similarity_score'] / 5) \
        for e in dataset]
    return samples

train_samples = samples_from_dataset(train_test_valid_dataset["train"])
dev_samples = samples_from_dataset(train_test_valid_dataset["valid"])
test_samples = samples_from_dataset(train_test_valid_dataset["test"])

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

initial_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, write_csv=False)
initial_evaluator(model)

logging.info("Read stsb-multi-mt dev dataset")
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))


## Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)


##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

#model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='stsb-multi-mt-test')
test_evaluator(model, output_path=model_save_path)


Some weights of BertModel were not initialized from the model checkpoint at Rajan/NepaliBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/293 [00:00<?, ?it/s]

Iteration:   0%|          | 0/293 [00:00<?, ?it/s]

Iteration:   0%|          | 0/293 [00:00<?, ?it/s]

Iteration:   0%|          | 0/293 [00:00<?, ?it/s]

0.6840061084455473

In [ ]:
import scipy


model = SentenceTransformer('/content/drive/MyDrive/nepali_embeddings_data/model/Rajan-NepaliBERT-2023-08-24_17-32-47')

sentences = ["मेरो नाम सिद्धार्थ हो","के तपाई काठमाडौं जाँदै हुनुहुन्छ?"]

sentence_embeddings = model.encode(sentences,normalize_embeddings=True)
print(sentence_embeddings)

queries = ["मेरो नाम सिद्धार्थ हो",'तिम्रो नाम सिद्धार्थ हो भन्ने सुनेको छु', 'तिमी सिद्धार्थ हौ भने', 'मलाई खेलकुद गर्न मन पर्छ', 'म काठमाडौं घुम्छु।', 'सिद्धार्थ तिमी काठमाडौं घुम्छौ।']

query_embeddings = model.encode(queries, normalize_embeddings=True)

for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]
    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])
    final_result = {}

    for index, distance in results:
        sentence = sentences[index].strip()
        score = 1 - distance
        final_result[sentence] = score
    print(final_result)

[[ 0.055093   -0.00638872 -0.09560401 ... -0.02701242 -0.01968973
  -0.0408456 ]
 [-0.08580937 -0.02025252 -0.0214393  ... -0.02066883  0.0486115
  -0.04172223]]
{'मेरो नाम सिद्धार्थ हो': 0.9999999999999394, 'के तपाई काठमाडौं जाँदै हुनुहुन्छ?': 0.23645517563345475}
{'मेरो नाम सिद्धार्थ हो': 0.47735958215086427, 'के तपाई काठमाडौं जाँदै हुनुहुन्छ?': 0.18993900272528674}
{'मेरो नाम सिद्धार्थ हो': 0.6557779605034821, 'के तपाई काठमाडौं जाँदै हुनुहुन्छ?': 0.2177350634945352}
{'मेरो नाम सिद्धार्थ हो': 0.23973909172749153, 'के तपाई काठमाडौं जाँदै हुनुहुन्छ?': -0.05870688677025382}
{'के तपाई काठमाडौं जाँदै हुनुहुन्छ?': 0.4837968927808206, 'मेरो नाम सिद्धार्थ हो': 0.21989072427402756}
{'के तपाई काठमाडौं जाँदै हुनुहुन्छ?': 0.4978743953120155, 'मेरो नाम सिद्धार्थ हो': 0.49118924641340067}


In [ ]:
# rajan
{'मेरो नाम सिद्धार्थ हो': 0.9999999999999394, 'के तपाई काठमाडौं जाँदै हुनुहुन्छ?': 0.23645517563345475}
{'मेरो नाम सिद्धार्थ हो': 0.47735958215086427, 'के तपाई काठमाडौं जाँदै हुनुहुन्छ?': 0.18993900272528674}
{'मेरो नाम सिद्धार्थ हो': 0.6557779605034821, 'के तपाई काठमाडौं जाँदै हुनुहुन्छ?': 0.2177350634945352}
{'मेरो नाम सिद्धार्थ हो': 0.23973909172749153, 'के तपाई काठमाडौं जाँदै हुनुहुन्छ?': -0.05870688677025382}
{'के तपाई काठमाडौं जाँदै हुनुहुन्छ?': 0.4837968927808206, 'मेरो नाम सिद्धार्थ हो': 0.21989072427402756}
{'के तपाई काठमाडौं जाँदै हुनुहुन्छ?': 0.4978743953120155, 'मेरो नाम सिद्धार्थ हो': 0.49118924641340067}

#shushanta dai bert
{'मेरो नाम सिद्धार्थ हो': 0.9999999999998942, 'के तपाई काठमाडौं जाँदै हुनुहुन्छ?': 0.11708155923018682}
{'मेरो नाम सिद्धार्थ हो': 0.6728865844299373, 'के तपाई काठमाडौं जाँदै हुनुहुन्छ?': 0.17031113257826402}
{'मेरो नाम सिद्धार्थ हो': 0.6218690943409064, 'के तपाई काठमाडौं जाँदै हुनुहुन्छ?': 0.04879205272340059}
{'मेरो नाम सिद्धार्थ हो': 0.29005286825170506, 'के तपाई काठमाडौं जाँदै हुनुहुन्छ?': 0.2831765231719281}
{'के तपाई काठमाडौं जाँदै हुनुहुन्छ?': 0.42450794113096924, 'मेरो नाम सिद्धार्थ हो': 0.14793860404784775}
{'मेरो नाम सिद्धार्थ हो': 0.4190390200026235, 'के तपाई काठमाडौं जाँदै हुनुहुन्छ?': 0.31644460869667246}